<h1> Thursday Harbor: SciPy Tutorial </h1>

In [ ]:
# Mandatory package imports
import scipy
import scipy.signal as signal
import scipy.ndimage as ndi
import scipy.stats as stats
import scipy.optimize as optimize

from PIL import Image
import numpy as np
import pandas as pd
import h5py

import matplotlib.pyplot as plt

# display initialization
%matplotlib inline 

In [ ]:
# Optional imports
import skimage.feature
import rpy2

In [ ]:
# Make sure all data files are available
import os
for f in ['MAX_ZSeries-ch1.tif', 'MAX_ZSeries-ch2.tif', 'noisy_signal.h5', 
          'power_calibrations.csv', 'sweeps.h5', 'BOb.png']:
    if not os.path.isfile(f):
        print 'please download %s' %(f)

<h2> Outline </h2>

1. Curve fitting and optimization using scipy.optimize
2. Signal Processing and 1D Filtering using scipy.signal
4. 2D (image) loading and filtering using scipy.ndimage and scipy.signal
4. Some statistics using scipy.stats
5. Optional material/exercise (time permitting)

Note that during all of these modules, there will be examples of various ways to load/save and visualize data

<h2> Curve Fitting & Optimization </h2>

Many of the functions we are interested in are nonlinear. A nonlinear least squares fit is an iterative process to minimize the error function (residuals) of the fit to the data provided. This is why a nonlinear least squares fit, unlike a regression, needs an initial guess for the parameters to be optimized. 

Let's start with a simple example: In laser microscopy, we control the power of a laser by sending it a command voltage (via a Pockels cell). As we increase the command voltage, the laser outputs more power. However, the voltage/power relationship is nonlinear, so we will attempt to find an analytic equation that describes this relationship by fitting a sine wave to the measured data. 

The data are saved as columns in a .csv file. We will use the Pandas library to quickly load this data and retain its tabular format.

In [ ]:
#load up the data file here
power_calib_file = 'power_calibrations.csv'
power_df = pd.read_csv(power_calib_file)
power_df.head()

The Pandas library also contains some "smart" (and quick) plotting functions, where we can tell the plot function which column of the dataframe we want on each axis. 

In [ ]:
power_df.plot(x='PockelVoltage', y='FidelityPowerPerArea', kind='scatter');

I've written two important functions below: 

* `sine(params, x)` generates a sine wave using 4 parameters that set the scale/offset of the wave, and either a numpy array or scalar of input data. We want to fit this function to our data above by determining the correct set of parameters. The return value(s) are given by the equation:

$$
f(x) = A  sin(B (x-C)) + D
$$


* `err_fxn(params, x, y)` defines the error function to be minized by SciPy's `least_squares` functions. It is simply the difference (residuals) of the measured data and sine wave function.

In [ ]:
def sine(params, x):
    """ Model function determines the shape of the curve to fit.
    """
    A, B, C, D = params
    return A * np.sin(B * (x-C)) + D

def err_fxn(params, x, y):
    """ Error function that returns the difference between the measured
    values and the predicted values given by sine(x)
    """
    return y - sine(params, x)


p0 = (30, 0.2, 0.5, 30) # initial guess

voltage_input = power_df.PockelVoltage.values # input to the fitting function
measured_power = power_df.FidelityPowerPerArea.values # real data used to find fit

SciPy's `optimize.least_squares` function takes two required inputs -- the error function and the initial parameters -- as well as any additional arguments that need to be passed into the error function in proper order [in our case, the input data (voltage_input) and the measured data (measured_power)]. 

In [ ]:
fit = optimize.least_squares(err_fxn, p0, args = (voltage_input, measured_power))

print fit.x

The fit parameters tell us our analytic equation should take the following form:

$$
f(x) = 32.7  sin(1.94 (x-0.79)) + 32.3
$$


Let's qualitatively see how good the parameter estimation is by passing the fit parameters into the sigmoid function and plotting it on the same axis as our measured results. 

In [ ]:
sine_fit = sine(fit.x, voltage_input) #pass our optimized fit parameters and input into the sigmoid function

In [ ]:
########### PLOTTING #############
fig = plt.figure()
ax = fig.add_subplot(111)
data = ax.plot(voltage_input, measured_power, 'o')
fit_data = ax.plot(voltage_input, sine_fit, color='r', linewidth=2)
plt.legend(['data', 'fit'])
ax.set_xlabel('Voltage')
ax.set_ylabel('mW/um2');
##################################

**SOME ADDITIONAL TIPS:**

If `optimize.least_squares` is not coverging on a reasonable solution, consider adjusting your initial parameter guess. A good way to do that would be to plot several options for each parameter and qualitatively look for a reaosnable fit to your data. Pass that guess into `least_squares` and see if a minimum can be found. 

Next, `optimize.least_squares` takes some additional arguments that can give you more control over the fitting. You can set bounds to constrain the minimization region, or the minimzation method. If computation time is an issue, the Jaocbian matrix (a matrix of the partial derivatives of each component of fit) can be defined and passed into `optimize.least_squares`. 

See more here: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html

Additionally, `optimize.minimize` has similar functionality, with more freedom to set various bounds and methods for optimization. `optimize.curve_fit` contains a built in error function, so it is not necessary to define this, but only the fit function, when using this module.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html<br>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

** EXERCISE: FIT AN EXPONENTIAL TO A PHOTOBLEACHED CALCIUM TRACE AND REMOVE THIS NONLINEAR BASELINE**

In [ ]:
################ EXERCISE ####################
# Let's load up a noisy trace using h5py
# Note that by using "with" instead of file.open(), the file is automatically closed once we are completed

with h5py.File('noisy_signal.h5', 'r') as f:
    noisy_ca_trace = np.array(f['signal']) #path to trace

sample_rate = 30 #Hz
step_size = 1.0 / sample_rate
time_vec = np.arange(len(noisy_ca_trace)) * step_size

An exponential should take the following form:

$$
Y = A*{e}^{Bt} + D
$$

Note that your initial guess for B should be negative for a decaying exponential.

In [ ]:
############### EXERCISE CONT #################
# Suggestions: 
# 1) write a function that takes a tuple of parameters and an array
#    of time values, and returns an exponential 
# 2) optimize your parameter guess using optimize.least_squares()
# 3) subtract the exponential fit from the trace 

def exponential(params, t):
    """define exp_fxn here"""
    return exp_fxn


<h2> Signal Processing </h2>

By digitally sampling a signal, we are turning a continuous signal into a discrete, finite signal, and thus introduce several limitations into how we approach extracting meaning from data. The most obvious is the sampling rate: we cannot expect to detect any periodic signal that occurs faster than half the sampling rate (Nyquist frequency) due to the potential for aliasing errors. The sampling rate thus plays a big role in the detecting frequencies in data and applying any desired filters. 

Fourier analysis is the cornerstone of DSP. It is predicated upon the theory that any signal can be written as a sum of an infinite number of cosines and sines of different frequency, amplitude, and phase. Applicable to signal analysis, is the ability to bring a signal into the Fourier domain, and see what frequencies are present in the signal (and at what powers). From there, we can try to figure out where to properly apply filters to attenuate any noise. 

The Fast Fourier Transform (FFT) is the algorithm most commonly used to perform this analysis on a finite, sampled signal. We will start by examining the FFT of the baseline subtracted trace we calculated in the above exercise.

In [ ]:
# If you did not manage to subtract the baseline from the calcium
# signal above, then you can load ours:
with h5py.File('noisy_signal.h5', 'r') as f:
    baseline_subtracted_ca = np.array(f['baseline_subtracted_signal'])
    

In [ ]:
# First, calculate the discrete fourier transform of the signal.
ft = scipy.fftpack.fft(baseline_subtracted_ca)

# Notice that the shape of this array is the same, but the dtype
# has changed:
print "shape: ", baseline_subtracted_ca.shape, " => ", ft.shape
print "dtype: ", baseline_subtracted_ca.dtype, " => ", ft.dtype

In [ ]:
# fft() returns an array of complex values; here we use abs(ft) to get  
# the _magnitude_, since we are only interested in the power of each 
# frequency, not the phase (and in any case, we can't plot complex
# numbers so we needed to convert them to real values somehow).
abs_ft = abs(ft)
print abs_ft.dtype

# Let's see what this array looks like:
plt.figure(figsize=(12, 5))
plt.plot(abs_ft);

In [ ]:
# Notice that the right half of the fourier transform is a mirror image
# of the left side. Since we started with an input array of real values
# (not complex), it is safe to discard this data:
pos_ft = abs_ft[:len(abs_ft)//2]

# If we imagine our original signal being composed of many different sine waves
# added together, then each value in abs_ft tells us the amplitude of a particular
# frequency of sine wave present in the original signal. 
# The fftfreq() function returns an array of the frequencies for each value in abs_ft:
freqs = scipy.fftpack.fftfreq(len(baseline_subtracted_ca), step_size)
pos_freqs = freqs[:len(abs_ft)//2]  # clip these values as well

plt.figure(figsize=(12, 5))
plt.plot(pos_freqs, pos_ft)
plt.xlabel('Frequency (Hz)');

So it looks like we have some low frequency noise in the 0-1Hz range, and high frequency noise above 4Hz. Our signal is buried in there somewhere (hopefully in the quiet 1-3Hz range).

---------------------------

Now that we know what frequency components exist in the signal, we can design filters to attenuate any specific frequencies we do not want, or broadband noise in general. There are numerous filter options, and choosing one is usually a trade off between computation time, ripple in the stopband/passband, and the desire for recursive/nonrecursive methods. In this example, we will implement the commonly used Butterworth filter, which has a flat pass band and approaches ideal as the order is increased. 

Most of SciPy's tools for processing 1D signal data are found in the `signal` subpackage:
https://docs.scipy.org/doc/scipy-0.16.1/reference/signal.html


If you would like more details on other filter options, the following is a great resource (although for MATLAB, analagous options exist in Python):

https://www.mathworks.com/help/signal/examples/practical-introduction-to-digital-filter-design.html

In [ ]:
# Lets design a low pass filter to reject high frequency noise above 5Hz
cutoff_freq = 5

# The behavior of our filter is determined by two parameters (b, a) which
# can be computed using the butter() function:

# The cutoff frequency must be mapped [0,1]; the highest cutoff possible (fs/2) is 1
low = cutoff_freq / (0.5 * sample_rate) 
# larger values for order create a sharper cutoff
order = 7  
b, a = signal.butter(order, [low], btype='low')

# With parameters (b, a), we can plot the frequency response of this filter 
w, h = signal.freqz(b, a)
filter_freq_axis  = 1 / (2 * step_size * np.pi) * w
filter_response = abs(h)

plt.figure(figsize=(12, 5))
plt.plot(pos_freqs, pos_ft / pos_ft.max())
plt.plot(filter_freq_axis, filter_response, 'red');
plt.xlabel('Frequency (Hz)');

In [ ]:
# Finally, we can filter our signal and see the result:
filtered_signal = signal.filtfilt(b, a, baseline_subtracted_ca) # *see note below

plt.figure(figsize=(12, 5))
plt.plot(time_vec, baseline_subtracted_ca)
plt.plot(time_vec, filtered_signal);
plt.xlabel('s');

In [ ]:
# Getting close! Let's see what the frequency spectrum looks like now:
filtered_ft = abs(scipy.fftpack.fft(filtered_signal))[:len(filtered_signal)//2]

plt.figure(figsize=(12, 5))
plt.plot(pos_freqs, pos_ft)
plt.plot(pos_freqs, filtered_ft);
plt.xlabel('Frequency (Hz)');

** *NOTE: ** There are in fact two commonly used functions to filter you signal: `signal.lfilter` and `signal.filtfilt`. The former applies the filter once, and therefore introduces a phase shift, while the latter does both a forward and backward filter application to eliminate the phase shift (but increases computation time)

-------------------

We saw in the above example what a filter looks like in the Fourier domain. In fact, the method of applying a filter differs in time and frequency. In the Fourier domain, we could imagine simply multiplying the signal and the filter, attenuating everything outside the passband, and retaining everything within the passband. This multiplication in frequency is actually a different operation -- convolution -- in time.  

** EXERCISE: FURTHER FILTER THE ABOVE SIGNAL**

In [ ]:
############### EXERCISE ###############
# suggestions: 
# 1) Adjust the lowpass cutoff frequency to reject the remaining high frequency noise
# 2) Implement a high pass filter to eliminate some low-frequency oscillations
#    (note: the final argument to the butter() function can be either 'low' or 'high')
# 3) Maybe there is a more clever way to remove the LF noise?

<h2> Image Processing & 2D Filtering</h2>

Let's start by loading in the red and green channels of a two photon image, and convincing ourselves that images are really just matrices of pixel values. 

In [ ]:
im_ch1 = ndi.imread('MAX_ZSeries-ch1.tif')
im_ch2 = ndi.imread('MAX_ZSeries-ch2.tif')
print im_ch1.shape, type(im_ch1), im_ch1.max(), im_ch1.min()

To create a colorful RGB image, we need to merge these two channels. To do that, let's create a numpy array full of zeros that has the same width and height as our channel arrays, but a depth of 3 (R, G, B). We can assign channel 1 (red) to the first stack, and channel 2 (green) to the second. Matplotlib has built in functions for displaying grayscale and rgb images.

In [ ]:
# How to merge color channels
rgb_im = np.zeros((im_ch1.shape[0], im_ch1.shape[1], 3), 'float32')
rgb_im[...,0] = im_ch1
rgb_im[...,1] = im_ch2

#For RGB images of float dtype, matplotlib requires values to be between 0-1; so we normalize to the max
rgb_im = rgb_im / rgb_im.max()

In [ ]:
########## DISPLAY ###########
fig_image = plt.figure(figsize=(15, 10))
ax1 = fig_image.add_subplot(131)
ax2 = fig_image.add_subplot(132)
ax3 = fig_image.add_subplot(133)

ax1.imshow(im_ch1, cmap=plt.cm.gray)
ax2.imshow(im_ch2, cmap=plt.cm.gray)
ax3.imshow(rgb_im) ;

**2D Filtering**

Image filtering often begins with the same approach as with 1D signals--we use some kind of low-pass filter to remove noise, and some kind of high-pass filter to remove the baseline. We could, in principle, apply the same tools we used above to filtering 2D image data: we can decompose the image into its frequency components and design filters with specific bandpass characteristics. However, is is often the case that with images, the interesting data shows up in the _phase_ rather than the _frequency_. Thus, we take a somewhat different approach to handling this type of signal.

Most of our tools for image processing in scipy are found in the `ndimage` subpackage:
https://docs.scipy.org/doc/scipy-0.16.1/reference/ndimage.html

-------------------------

First, let's see if we can clean up the noisy green channel. A Gaussian filter will smooth the data in 2D (much like a running average), whereas a median filter will eliminate some salt-and-pepper noise while preserving the integrity of edges:

In [ ]:
# Zoom in on a small region so we can clearly see the noise
chunk = im_ch2[100:200, 100:200]

# Use gaussian_filter to smooth out the image
smooth = ndi.gaussian_filter(chunk, sigma=2)

# Use median_filter to remove speckle but preserve sharp edges
median = ndi.median_filter(chunk, size=3, mode='constant')

In [ ]:
############### PLOTTING ################
fig_image = plt.figure(figsize=(15, 10))
ax1 = fig_image.add_subplot(131)
ax2 = fig_image.add_subplot(132)
ax3 = fig_image.add_subplot(133)
ax1.imshow(chunk, cmap=plt.cm.gray)
ax1.set_title('Original')
ax2.imshow(smooth, cmap=plt.cm.gray)
ax2.set_title('Gaussian Smoothing')
ax3.imshow(median, cmap=plt.cm.gray)
ax3.set_title('Median Filter');
##########################################

In [ ]:
############### EXERCISE ###############
# Selecting an appropriate filter size is important
# for ensuring that you reject as much noise as possible without
# losing too much detail.

# Try changing the gaussian and median filter sizes to get an 
# intuitive sense of this tradeoff. 

**Baseline Removal**

In [ ]:
cell_im = ndi.imread('BOb.png')[...,0] #load image

plt.figure(figsize=(7, 7))
plt.imshow(cell_im, cmap='gray');

Notice that some regions of this image are brighter than others; how can we flatten the image? Similar to our approach with the 1D calcium signal, we can use a heavy low-pass filter to get a profile of the background, and then subtract or divide it from the image:

In [ ]:
background = ndi.gaussian_filter(cell_im.astype(float), sigma=20)

plt.figure(figsize=(7, 7))
plt.imshow(background, cmap='gray');

In [ ]:
# Divide the background from the original image. 
# Using division effectively amplifies the signal in the dark regions of the
# image, causing all cells to appear roughly equally bright
bg_removed = cell_im / background

################# PLOTTING ##################
fig_image = plt.figure(figsize=(20, 15))
ax1 = fig_image.add_subplot(131)
ax2 = fig_image.add_subplot(132)
ax1.imshow(cell_im, cmap=plt.cm.gray)
ax2.imshow(bg_removed, cmap=plt.cm.gray);

In [ ]:
############### EXERCISE ###############
# Try experimenting with different filter sizes in the background
# removal process. What features of the image do you trade by
# increasing or decreasing the size?

This image is pretty well cleaned up at this point. Let's say our goal now is to find regions of interest corresponding to individual cells. This can be a surprisingly difficult problem, and there are many ways to approach it. One simple way looks like:

1. Manually select a threshold brightness that separates cells from background
2. Find all contiguous regions "blobs" that are above threshold

In [ ]:
threshold = bg_removed.mean() + bg_removed.std()
binary_img = bg_removed > threshold

plt.figure(figsize=(7, 7))
plt.imshow(binary_img);

In [ ]:
# That's a start, but perhaps we can clean it up a bit?
clean_bin_img = ndi.median_filter(binary_img, 10)

plt.figure(figsize=(7, 7))
plt.imshow(clean_bin_img);

In [ ]:
# Now we create a "label" array: the region over each 
# cell is given a unique integer value
labels = scipy.ndimage.label(clean_bin_img)[0]

plt.figure(figsize=(7, 7))
plt.imshow(labels);

In [ ]:
# From the label array, we can select individual cells from the original image:
cells = [bg_removed[i, j] for i, j in ndi.find_objects(labels)]

plt.imshow(cells[16], cmap='gray');

<h2> Statistics </h2>

The `scipy.stats` module contains two major parts:

1. A set of classes representing [many continuous and discrete statistical distributions](https://docs.scipy.org/doc/scipy/reference/stats.html#continuous-distributions) (gaussian, binomial, poisson, etc.)
2. A set of functions implementing [common statistical methods](https://docs.scipy.org/doc/scipy/reference/stats.html#statistical-functions) such as t-tests, ANOVA, correlation corfficients, etc.

---------------------

Let's begin by generating some random data. Scipy's distribution classes are designed such that each one implements the same basic interface (to the extent that this is possible). For example, we can generate random variables from any distribution by using its `rvs()` method:

In [ ]:
import scipy.stats as stats
import numpy as np

# Pick 500 random values from a normal distribution centered at 5.0
dist1 = stats.norm(loc=5.0, scale=1.2)
data1 = dist1.rvs(size=500)

# Pick 500 random values from a lognormal distribution
dist2 = stats.lognorm(s=1, scale=3.2)
data2 = dist2.rvs(size=500)

# as a reminder, numpy already provides some very basic tools:
print "data1 mean: %f  stdev: %f" % (data1.mean(), data1.std())
print "data2 mean: %f  stdev: %f" % (data2.mean(), data2.std())

In [ ]:
from matplotlib.pylab import axes, figure

In [ ]:
# plot histograms of the random values we generated
ax = axes()
bins = np.linspace(0, 15, 40)
ax.hist(data1, bins=bins, color=(1.0, 0.6, 0.2, 0.3));
ax.hist(data2, bins=bins, color=(0.2, 0.3, 1.0, 0.3));

# Use the pdf() method to plot the probability density function of each distribution
x = np.linspace(0, 15, 200)
ax.plot(x, dist1.pdf(x) * 200, color=(1, 0.6, 0.2));
ax.plot(x, dist2.pdf(x) * 200, color=(0.2, 0.3, 1.0));

We just saw uses of the `rvs()` and `pdf()` methods of continuous distributions. The full list of methods available for each distribution can be found at the bottom of its documentation page. Take a moment to look at the documentation for [scipy.norm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html#scipy.stats.norm) to get an idea of the methods available.

-----------------

Now let's try out a few of the functions available in scipy.stats. Start by having a peek at the [documentation](https://docs.scipy.org/doc/scipy/reference/stats.html#statistical-functions).

In [ ]:
# Imagine data1 and data2 are repeated measurements taken under different conditions.
# It sure looks like the results are different, but our reviewer is asking for a p-value.
# How about a T-test?
stats.ttest_ind(data1, data2)

In [ ]:
# Let's try a different approach..
stats.ks_2samp(data1, data2)

In [ ]:
############### EXERCISE ###############
#
# Generate two random data sets and calculate the Pearson
# correlation coefficient between them. How many comparisons do
# you need to make before getting a false positive?
#
#  See: scipy.stats.norm and scipy.stats.pearsonr

<h2> Additional Material & Exercises </h2>

### Using R statistical tools from Python

Some of the highest quality, vetted statistical tools are written in R. The `rpy2` module makes it possible to call R functions from the comfort of your Python environment.

In [ ]:
# to run this code, you need the rpy2 package 
# (this can be installed by running `conda install rpy2`)
import rpy2.robjects.packages

In [ ]:
# install Jewell & Witten's LZeroSpikeInference package (if needed)
import rpy2.robjects.packages as rpackages
if not rpackages.isinstalled('LZeroSpikeInference'):
    url = 'https://github.com/jewellsean/LZeroSpikeInference/archive/master.zip'
    print("Downloading %s..." % url)
    import urllib2
    response = urllib2.urlopen(url)
    open('LZeroSpikeInference.zip', 'w').write(response.read())
    
    print("Extracting zip...")
    import zipfile
    zip_ref = zipfile.ZipFile('LZeroSpikeInference.zip', 'r')
    zip_ref.extractall('LZeroSpikeInference')
    zip_ref.close()

    print("Installing package to R...")
    
    utils = rpackages.importr('utils')
    utils.install_packages("./LZeroSpikeInference/LZeroSpikeInference-master", repo=rpy2.robjects.NULL, type="source")
    
    print("Done!")

In [ ]:
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
lzsi = rpy2.robjects.packages.importr("LZeroSpikeInference")

In [ ]:
# If you did not manage to fully filter the ca signal,
# then you can load ours:
with h5py.File('noisy_signal.h5', 'r') as f:
    clean_ca_signal = np.array(f['clean_ca_signal'])

In [ ]:
data = clean_ca_signal[5000:6000] / clean_ca_signal[5000:6000].max() + 0.1
fit = lzsi.estimateSpikes(data, **{'gam':0.97, 'lambda':1, 'type':"ar1"})

In [ ]:
ax = axes((0, 0, 1.4, 1))
ax.plot(data, label='data')
ax.plot(fit[1], label='l0 estimate')
ax.legend();

### Extracting events from electrophysiology trace

In [ ]:
# Let's load up a slightly more complicated signal
# This trace has both synaptic events and background depolarization due to an optogenetic stimulation

with h5py.File('sweeps.h5', 'r') as f:
    electrode_2 = np.array(f['acquisition/timeseries/data_00000/electrode_2'])
    electrode_2_unit = f['acquisition/timeseries/data_00000/electrode_2'].attrs['unit']
    time = np.array(f['acquisition/timeseries/data_00000/time'])
    time_unit = f['acquisition/timeseries/data_00000/time'].attrs['unit']
    
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
ax.plot(time, electrode_2)
ax.set_xlabel(time_unit), ax.set_ylabel(electrode_2_unit);

In [ ]:
################ EXERCISE ###############
# Here are some tools

# take median of first 500 elements and subtract from trace
def zero_baseline(data, n=500):
    med = np.median(data[0:n])
    return data - med

# filter: this function will function as a low, high, or band pass filter, but you will 
# most likely want to use it as a low pass filter for the signal above 
def filter_signal(data, sample_rate, cutoff_low = None, cutoff_high = None, order=5):
    if cutoff_low is not None:
        l = cutoff_low/(2*sample_rate)
        if cutoff_high is not None:
            filt_type = 'band'
            h = cutoff_high/(2*sample_rate)
            
            filts = [l, h]
        else:
            filt_type = 'low'
            filts = [l]
    elif cutoff_high is not None:
        filt_type = 'high'
        h = cutoff_high/(2*sample_rate)
        filts = [h]
    else:
        return 'Need some type of filter!'
    
    b, a = signal.butter(order, filts, btype = filt_type)
    
    filtered_data = signal.lfilter(b, a, data)
    
    return filtered_data
    

# sliding maximum filter to remove direct depolarization
def sliding_max(data, win_len, smooth_win=500):
    max_filt = ndi.maximum_filter(data, win_len)
    smoothed = np.convolve(max_filt, np.ones(smooth_win)/smooth_win, mode='same')
    return smoothed
           

# peak detection, takes in optional threshold arguments
def find_peaks(data, pos_threshold = None, neg_threshold = None):
    
    if pos_threshold is None:
        pos_threshold = np.mean(data) + 2.5*np.std(data)
    if neg_threshold is None:
        neg_threshold = np.mean(data) - 2.5*np.std(data)
        
    sig_deriv = np.diff(data)
    
    sig_deriv = (sig_deriv > 0).astype(np.byte) 
    
    neg_peak_opts = np.argwhere(sig_deriv[1:] - sig_deriv[:-1] == 1) 
    pos_peak_opts = np.argwhere(sig_deriv[1:] - sig_deriv[:-1] == -1)
    
    pos = pos_peak_opts[data[pos_peak_opts] > pos_threshold]
    neg = neg_peak_opts[data[neg_peak_opts] < neg_threshold]
    
    return pos, neg


############# EXERCISE ###############
## use the above functions and suggestions below to play with various filtering options
## here is a start
sample_rate = 1/(time[1] - time[0]) #get sampling rate from time vector

zeroed = zero_baseline(electrode_2) #start by zeroing the baseline
filtered = filter_signal(zeroed, sample_rate, cutoff_low = 1000) #add some filtering

win = 3000 #adjust this to change smoothing
max_filter = sliding_max(filtered, win)

depolarization_subtracted = filtered[0:len(max_filter)] - max_filter

pos, neg = find_peaks(depolarization_subtracted)
####################################


#### plotting
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
ax.plot(time[0:len(depolarization_subtracted)], depolarization_subtracted)
ax.plot(time[0:len(max_filter)], max_filter)
ax.legend(['filtered', 'maximum filter'])
ax.scatter(time[neg], depolarization_subtracted[neg], color='r')
ax.scatter(time[pos], depolarization_subtracted[pos], color='g')
ax.set_xlabel('s')
ax.set_ylabel('mV');

The scikit-image packages contains several "blob detection" modules, and we will employ two of them here to attempt to classify cells in the above image:

1. Difference of Gaussians: this method involves the subtraction of two gaussian filters (similar to the ones employed earlier). This is analagous to passband in signal processing, and in an image, will extract the edges of an object. 

2. Laplacian of Gaussians: A gaussian filter first acts to smooth the image, and a Laplacian (which takes the second derivative, $L_{xx} + L_{yy} $) performs edge detection, but with a positive response moving into the blob, and a negative response moving out of the blob. 

Read more here:

http://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.blob_dog

http://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.blob_log

http://fourier.eng.hmc.edu/e161/lectures/gradient/node9.html

http://fourier.eng.hmc.edu/e161/lectures/gradient/node8.html

### Alternative approaches to detecting cell ROIs


If our eventual goal is to find regions of interest corresponding to cells, one challenging factor may be the apparent difference in morphology between the cells in the images above. You will notice that some are filled (solid) while others appear to be rings. We can take advantage of a morphological operation, dilation, to essentially "fill" all rings. In dilation, each pixel value is replaced with the maximum pixel value of the region around it. This will act to both enlarge and fill the circle. The extent of this is determined by the size of the structuring element used for dilation. Although this operation will fill rings (good), it will artifically enlarge both cells and noise (bad). We can subsequently perform an erosion step with a structuring element of the same size. Erosion replaces each pixel with the minimum pixel value of its surroundings. Note that erosion is not quite the inverse of dilation: the rings that we filled will remain closed (they are surrounded by a neighborhood or bright pixels), but the size of the object will decrease. Both these operations can be done in ndimage with a single function: `grey_closing`

http://utam.gg.utah.edu/tomo03/03_mid/HTML/node119.html

In [ ]:
# Let's use our background-subtracted image fom above
r = 5
structure = np.fromfunction(lambda i,j: (i-r)**2+(j-r)**2 < r**2, (2*r+1, 2*r+1))
closing_3 = ndi.morphology.grey_closing(bg_removed, structure=structure)
closing_10 = ndi.morphology.grey_closing(bg_removed, size=(10,10))

In [ ]:
################# PLOTTING ##################
fig_image = plt.figure(figsize=(15, 10))
ax1 = fig_image.add_subplot(131)
ax2 = fig_image.add_subplot(132)
ax3 = fig_image.add_subplot(133)

ax1.imshow(bg_removed, cmap=plt.cm.gray)
ax2.imshow(closing_3, cmap=plt.cm.gray)
ax3.imshow(closing_10, cmap=plt.cm.gray)

ax1.set_title('original')
ax2.set_title('closing: size 3')
ax3.set_title('closing: size 10');

In [ ]:
DoG = skimage.feature.blob_dog(closing_10, max_sigma=10, threshold=2) 
LoG = skimage.feature.blob_log(closing_10, max_sigma=10, threshold=0.5)
# play around with optional parameters, max_sigma & threshold, to alter the size/brightness
# of the blobs detected

fig_image = plt.figure(figsize=(15, 10))
ax1 = fig_image.add_subplot(121)
ax2 = fig_image.add_subplot(122)


ax1.imshow(cell_im) #back to our original images
ax2.imshow(cell_im)


for roi_DoG in DoG: 
    y_DoG = roi_DoG[0]
    x_DoG = roi_DoG[1]
    rad_DoG = roi_DoG[2] * 2**0.5
    
    ##### EXERCISE ########
    
    # filter by size
    #if rad > threshold_pixels:
        #color = 'r'
        
    #else:
        #color = 'g'
        
    ax1.add_patch(plt.Circle((x_DoG,y_DoG), rad_DoG, color='r', fill=0))

for roi_LoG in LoG:
    y_LoG = roi_LoG[0]
    x_LoG = roi_LoG[1]
    rad_LoG = roi_LoG[2] * 2**0.5
    
    ##### EXERCISE ########
    
    # filter by size
    # if rad > threshold_pixels:
        # color = 'r'
        
    # else:
        # continue
    
    
    ax2.add_patch(plt.Circle((x_LoG,y_LoG), rad_LoG, color='r', fill=0))


ax2.set_title('Laplacian of Gaussian')
ax1.set_title('Difference of Gaussian')